In [2]:
import praw
import math
import pandas as pd
from datetime import datetime
import re
import time
from tqdm import tqdm
tqdm.pandas()

user_agent="agent01"
reddit = praw.Reddit(username="No-Professional-8030",
                     password="Tutorial01",
                     client_id="2MS-Ef8sTpSYzdbqEpIxAg",
                     client_secret="Cnht4sjg8xG0EZTaPI8Yk0Kn3zGKqw",
                     user_agent=user_agent
)
subreddit_name="buildapcsales"
subreddit = reddit.subreddit(subreddit_name)
print(subreddit.display_name)

hot_subreddit=subreddit.hot()

df = pd.DataFrame()
post=[]
type = ''
price = 0.00
item = ''
regex= r"\[([^\]]+)\](.+?)(\$\d+)"
for submission in subreddit.hot():
    if(submission.id in ('yvocv1','z2atkk')):
        continue
    result = re.search(r'\[([^\]]+)\](.+?)(\$\d+\.?\d+)', submission.title)
    type=result.group(1)
    item=result.group(2).lstrip()
    price=result.group(3)
#     print(type)
#     print(item)
#     print(price)
    post.append([submission.id,type,item,price,submission.score,submission.url,submission.created])

post = pd.DataFrame(post,columns=['id','Type','Item','Price','score','url','Time Created'])

def get_comments(submission_id):
    submission = reddit.submission(submission_id)
    comments = []
    for top_level_comment in submission.comments:
        comments.append(top_level_comment.body)
        
    comment = ' '.join(comments)
    comment = comment.replace("\n",' ')
    return comment

post['Comment'] = post['id'].progress_apply(lambda x: get_comments(x))

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
def sentiment_scores(sentence):

    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)
    
    if sentiment_dict['compound'] >= 0.05 :
        return "Positive"
    elif sentiment_dict['compound'] <= - 0.05 :
        return "Negative"
    else :
        return "Neutral"

post['Sentiment'] = post['Comment'].progress_apply(lambda x: sentiment_scores(x))

def get_date_time(x):
    return datetime.fromtimestamp(x)#.strftime('%Y-%m-%d')

post['Time Created1'] = post['Time Created'].progress_apply(lambda x: get_date_time(x))

def get_post_hours(x):
    return (time.time() - x)/3600

post['LikesPerHour'] = post.progress_apply(lambda x: math.ceil(x['score']/get_post_hours(x['Time Created'])),axis=1)

post.rename(columns={'score':'Overall Likes'},inplace=True)

post.drop(['Time Created'],axis=1,inplace=True)

post.rename(columns={'Time Created1':'Post Datetime'},inplace=True)

n = int(input("How many likes per hour ratio does the post have: "))
item = input("What type of sale item are they looking for: ")

post = post[post['LikesPerHour']>=n]

def search_item(x):
    return any([True if i.lower() in x.lower() else False for i in item.split()])

post = post[post['Item'].apply(lambda name: search_item(name))]

post.to_csv('post.csv',index=False) # use this post.csv to load into google sheet

buildapcsales


100%|███████████████████████████████████████████████████████████████████████████████| 98/98 [01:19<00:00,  1.24it/s]
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\avitr\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
100%|████████████████████████████████████████████████████████████████████████████| 98/98 [00:00<00:00, 32312.07it/s]


How many likes per hour ratio does the post have: 7
What type of sale item are they looking for: Amazon


# Google Sheet

In [ ]:
# create your client id, follow this link
#https://docs.gspread.org/en/latest/oauth2.html#for-end-users-using-oauth-client-id

In [ ]:
import gspread
gc = gspread.authorize(credentials)
# Read CSV file contents
content = open('post.csv', 'r').read()

gc.import_csv('<SPREADSHEET_ID>', content)

# or

https://towardsdatascience.com/using-python-to-push-your-pandas-dataframe-to-google-sheets-de69422508f

In [ ]:
# https://stackoverflow.com/questions/62917910/how-can-i-export-pandas-dataframe-to-google-sheets-using-python